### 그래픽 카드 설정 및 확인
- 현재 kernal에서 버전 확인이 꼭 필요한 것들 확인
- torch 버전이 2.5.1+cu121이 아닐 경우 아래의 주석 해제 후 실행

In [1]:
import torch
print(torch.__version__)       # 2.5.1
print(torch.version.cuda)      # 12.1
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.5.1+cu121
12.1
True
NVIDIA GeForce RTX 4050 Laptop GPU


CUDA 확인

In [2]:
import torch

print(torch.cuda.get_device_name(0))

#!nvidia-smi

NVIDIA GeForce RTX 4050 Laptop GPU


## 성능 향상을 위한 함수
#### Text 정제
- huggingface에서 개발자가 공개한 성능을 높이기 위한 데이터 전처리 과정입니다. 특수문자 및 이모지 등을 제거합니다
- 불용어는 제거하지 않습니다. 감정 모델에서는 불용어 또한 중요한 데이터일 수 있습니다.

In [3]:
import re
import emoji
from soynlp.normalizer import repeat_normalize

emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

### 텍스트 정제 확인

In [4]:
sentence = "나는 오늘 선배한테 혼났어 ★."
cleaned_sentence = clean(sentence)
print(cleaned_sentence)

나는 오늘 선배한테 혼났어 .


#### 감정 매핑.

In [5]:
# 감정 카테고리 (모델의 class 순서)
CATEGORIES = [
    "happy",
    "embarrass",
    "anger",
    "unrest",
    "damaged",
    "sadness",
]

# 이건 실제 csv에서 뽑아올 것.
label_map = {
    "기쁨": 0,     # happy
    "당황": 1,     # embarrass
    "분노": 2,     # anger
    "불안": 3,     # unrest
    "상처": 4,     # damaged
    "슬픔": 5,     # sadness
}

NUM_LABELS = len(CATEGORIES)

# 추론

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import os
import time

# 모델이랑 토크나이저 불러오기.
MODEL_DIR = r"C:\Users\SSAFY\Desktop\WANG\S13P31A106\ai\Classifier_Model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")

# 감정 카테고리 - 맨 위에 저장해놓은 그거
CATEGORIES = ["happy", "embarrass", "anger", "unrest", "damaged", "sadness"]

# 추론 함수. 여기에 txt를 넣으면 결과가 나옴.
def predict_emotion(texts):
    if isinstance(texts, str):
        texts = [texts]

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=64,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = F.softmax(outputs.logits, dim=-1)
        preds = probs.argmax(dim=-1)

    results = []
    for i, text in enumerate(texts):
        results.append({
            "text": clean(text),
            "pred_label": CATEGORIES[preds[i].item()],
            "probabilities": {CATEGORIES[j]: round(probs[i][j].item(), 4) for j in range(len(CATEGORIES))}
        })
    return results

# 넣을 문장 예시. 나중에는 사용자의 다이어리 내용이 들어갈 것.
sample_texts = [
    "오늘은 기분이 너무 좋아!",
    "화가 나서 견딜 수가 없어.",
    "너무 불안하고 초조해.",
    "나 기분이 이상해",
]

a = time.time()
results = predict_emotion(sample_texts)  # 전체 예측
b = time.time()

print(f"전체 걸린 시간: {b - a:.3f}초")
print(f"문장당 평균: {(b - a)/len(results):.3f}초")

for r in results:
    print(f"문장: {r['text']}")
    print(f"예측 감정: {r['pred_label']}")
    print("확률 분포:", r["probabilities"])
    print("-" * 60)


전체 걸린 시간: 0.019초
문장당 평균: 0.005초
문장: 오늘은 기분이 너무 좋아!
예측 감정: happy
확률 분포: {'happy': 0.9954, 'embarrass': 0.0008, 'anger': 0.0007, 'unrest': 0.0016, 'damaged': 0.0005, 'sadness': 0.001}
------------------------------------------------------------
문장: 화가 나서 견딜 수가 없어.
예측 감정: anger
확률 분포: {'happy': 0.0008, 'embarrass': 0.0146, 'anger': 0.9241, 'unrest': 0.0123, 'damaged': 0.0249, 'sadness': 0.0233}
------------------------------------------------------------
문장: 너무 불안하고 초조해.
예측 감정: unrest
확률 분포: {'happy': 0.0016, 'embarrass': 0.0125, 'anger': 0.0288, 'unrest': 0.9343, 'damaged': 0.0179, 'sadness': 0.0049}
------------------------------------------------------------
문장: 나 기분이 이상해
예측 감정: unrest
확률 분포: {'happy': 0.0068, 'embarrass': 0.0404, 'anger': 0.0795, 'unrest': 0.3857, 'damaged': 0.2655, 'sadness': 0.2221}
------------------------------------------------------------
